In [332]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('employee.csv')
df2 = pd.read_csv('sales_data.csv')

df2.rename(columns={'id ':'id','policy no':'policyno'},inplace=True)


In [333]:
df1['age']=(pd.Timestamp('now')-pd.to_datetime(df1['dob'],dayfirst=True))//np.timedelta64(1,'Y')
df1['tenure (in days)']=(pd.Timestamp('now')-pd.to_datetime(df1['Joining_date'],dayfirst=True))//np.timedelta64(1,"D")

df1['no. of policies']=df2.groupby("id",as_index=False)["id"].count()

df1 = pd.merge(df1, df2[df2['status'] == 'In Force'].groupby('id').size().reset_index(name='no_of_active_policy'), on='id', how='left')
df1 = pd.merge(df1, df2[df2['status'] == 'Pending'].groupby('id').size().reset_index(name='no_of_pending_policy'), on='id', how='left').fillna(0)
df1 = pd.merge(df1, df2[df2['status'] == 'Cancelled'].groupby('id').size().reset_index(name='no_of_cancelled_policy'), on='id', how='left').fillna(0)

df1 = pd.merge(df1,df2.groupby("id",as_index=False)["income"].mean(),on="id",how="inner").rename(columns={'income':'avg income'})

df1['no_of_pending_policy']=df1['no_of_pending_policy'].astype(int)
df1['no_of_cancelled_policy']=df1['no_of_cancelled_policy'].astype(int)
    

In [335]:
df1

,id,department,first,last,dob,Joining_date,age,tenure (in days),no. of policies,no_of_active_policy,no_of_pending_policy,no_of_cancelled_policy,avg income
0,1,Sales,Deepti,Sharma,01/01/1998,20/02/22,25,362,8,3,4,1,561890.375000
1,2,Sales,Suresh,Mishra,28/05/1992,31/05/17,30,2088,6,2,4,0,433256.666667
2,3,Sales,Maya,patil,01/02/1998,20/07/22,25,212,9,3,4,2,663854.555556
3,4,Sales,Prashant,Jadhav,27/08/1989,01/12/14,33,3000,5,1,2,2,641406.200000
4,5,Sales,Deepak,Kumar,01/02/1998,04/04/21,25,684,3,1,1,1,636665.000000
5,6,Sales,Pratik,Kumar,28/05/1993,05/08/15,29,2753,4,2,1,1,726671.500000
6,7,Sales,Mayur,Sinha,01/03/1998,09/07/22,24,223,1,1,0,0,454159.000000
7,8,Sales,Amit,Sharma,26/08/1985,10/09/09,37,4908,4,1,2,1,388765.250000
8,9,Sales,Prajakta,patil,12/03/1981,18/05/06,41,6119,6,1,4,1,648181.500000
9,10,Sales,Smita,Raina,28/05/1994,12/08/16,28,2380,5,2,1,2,857388.600000


In [ ]:

'''
NOTE :
1> Be careful while directly adding columns from one df to other. It works well only when you are sure that output size will be same as one of the existing columns (like age from DOB)
directly adding columns without any condition may create errors just like the pending and cancelled will do in the following lines: 
df1['no. of Active policies']=df2[df2["status"]=="In Force"].groupby("id",as_index=False,dropna=False)["id"].count() 
df1['no. of Pending policies']=df2[df2["status"]=="Pending"].groupby("id",as_index=False,dropna=False)["id"].count()
df1['no. of Cancelled policies']=df2[df2["status"]=="Cancelled"].groupby("id",as_index=False,dropna=False)["id"].count()


2> pd.to_datetime(df1['dob'],dayfirst=True) -> dayfirst=True is important when the pd.datetime when autoreads cannot decide if the value is m/d/y or d/m/y (ignoring this creates wrong data!)
'''

#rename(column={},inplace=True)           -> inplace=True doesn't require you to store DF somewhere, directly write the function and it works.